In [1]:
###Libs importeren
import requests
from bs4 import BeautifulSoup as bs

import pandas as pd

import redis
#json dumps
import json

In [2]:
##ik heb geprobeerd om zoveel mogenlijk in OO te werken

In [3]:
##Code scripting vorig jaar hergebruiken
def htmlFilter(URL):
    page = requests.get(URL)
    soup = bs(page.content)
    
    #classes voor zowel de nummers als de hashes zoeken
    results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})
    
    return results_numbers, results_hash

In [4]:
##Op youtube raden ze me aan om een dataframe in pandas te maken, dit hebben we geleerd in data science en data vis
def pandaDF(hash, time, BTC, USD):
    dataF = {'_id' : hash, 'Time' : time, 'BTC-value' : BTC, 'USD-value' : USD}
    frameUpdate = pd.DataFrame(dataF)
    
    #altijd eerst wat data cleanen doen
    frameUpdate['BTC-value'] = frameUpdate['BTC-value'].map(lambda x: x.rstrip(' BTC'))
    frameUpdate['BTC-value'] = frameUpdate['BTC-value'].str.replace(r'\,', '')
    frameUpdate['BTC-value'] = frameUpdate['BTC-value'].astype('float')
    
    frameUpdate['USD-value'] = frameUpdate['USD-value'].map(lambda x: x.lstrip('$'))
    frameUpdate['USD-value'] = frameUpdate['USD-value'].str.replace(r'\,', '')
    frameUpdate['USD-value'] = frameUpdate['USD-value'].astype('float')
    
    
    return frameUpdate


In [5]:
def tussenArrays(numbers, hash):
    ### heb gedubbelchecked en de values staan steeds op dezelfde plek in de array numbers
    
    only_time =[]
    only_BTC =[]
    only_USD=[]
    only_hash = [content.get_text() for content in hash]
    
    for i in range(len(numbers)):
        if(i%3==0):
            only_time.append(numbers[i].get_text())
        if(i%3==1):
            only_BTC.append(numbers[i].get_text())
        if(i%3==2):
            only_USD.append(numbers[i].get_text())
            
    return only_hash, only_time, only_BTC, only_USD

In [6]:
def zoeken(URL):
    ##drie vorige functies nu samen uitvoeren
    ##eerst de html zoeken vervolgens alle data in de juiste arrays steken om ten slotte ze in een pandas dataframe te steken
    #HTML-doorzoeken en alles ophalen
    numbers, hash = htmlFilter(URL)

    #Hash time btc usd in arrays steken
    only_hash, only_time, only_BTC, only_USD = tussenArrays(numbers, hash)

    #Make dataframe
    frame = pandaDF(only_hash, only_time, only_BTC, only_USD)
    
    return frame

In [7]:
##Redis verbinding
redisCon = redis.Redis(host="localhost", port=8081)
redisCon.flushdb()

True

In [ ]:
##stackoverflow
while True:
        URL = "https://www.blockchain.com/btc/unconfirmed-transactions"
        frame_new = zoeken(URL)
        
        js = frame_new.to_dict(orient="records")
        
        for element in js: 
            redisCon.set(element["_id"], json.dumps(element), ex=210)

C:\Users\32474\AppData\Local\Temp/ipykernel_24080/4205679505.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  frameUpdate['BTC-value'] = frameUpdate['BTC-value'].str.replace(r'\,', '')
C:\Users\32474\AppData\Local\Temp/ipykernel_24080/4205679505.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  frameUpdate['USD-value'] = frameUpdate['USD-value'].str.replace(r'\,', '')
